In [1]:
import boto3
import os, glob
import pandas as pd

from io import StringIO, BytesIO 
from datetime import datetime, timedelta

In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [3]:
bucket_obj_1 = bucket.objects.filter(Prefix='2022-03-10')
bucket_obj_2 = bucket.objects.filter(Prefix='2022-03-11')

objects = [obj for obj in bucket_obj_1] + [obj for obj in bucket_obj_2]
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2022-03-10/2022-03-10_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [4]:
# first record
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)

df_init = pd.read_csv(data, delimiter = ',')
df_init

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades


In [5]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [6]:
df_all = pd.DataFrame(columns=df_init.columns)

for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')    
    data = StringIO(csv_obj)
    
    df = pd.read_csv(data, delimiter=',')   
    df_all = df_all.append(df, ignore_index=True)

In [7]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-03-10,08:00,12.240,12.240,12.240,12.240,5316,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-03-10,08:00,39.000,40.180,38.580,39.880,6489,37
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-03-10,08:00,16.800,16.990,16.800,16.990,20802,27
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-03-10,08:00,200.900,202.000,200.800,201.500,2727,38
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-03-10,08:00,35.090,35.100,35.090,35.100,4470,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259966,DE000HAG0005,HAG,HENSOLDT AG INH O.N.,Common stock,EUR,5645114,2022-03-11,16:44,23.800,23.800,23.800,23.800,97,1
259967,DE000TUAG000,TUI1,TUI AG NA O.N.,Common stock,EUR,2504855,2022-03-11,16:45,2.757,2.757,2.757,2.757,3500,1
259968,IE000WF4FCJ3,XMQQ,HETF-FMNXFIE ETFDLA,ETF,EUR,7260915,2022-03-11,16:46,5.157,5.157,5.157,5.157,0,1
259969,LU1459802754,UIMB,UBSLFS-BB TIPS 10+ ADDL,ETF,EUR,2506174,2022-03-11,16:52,15.905,15.905,15.905,15.905,64,1


In [8]:
# now only taking nesscary columns

columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-03-10,08:00,12.240,12.240,12.240,12.240,5316
1,DE000A0DJ6J9,2022-03-10,08:00,39.000,40.180,38.580,39.880,6489
2,DE000A0D6554,2022-03-10,08:00,16.800,16.990,16.800,16.990,20802
3,DE000A0D9PT0,2022-03-10,08:00,200.900,202.000,200.800,201.500,2727
4,DE000A0HN5C6,2022-03-10,08:00,35.090,35.100,35.090,35.100,4470
...,...,...,...,...,...,...,...,...
259966,DE000HAG0005,2022-03-11,16:44,23.800,23.800,23.800,23.800,97
259967,DE000TUAG000,2022-03-11,16:45,2.757,2.757,2.757,2.757,3500
259968,IE000WF4FCJ3,2022-03-11,16:46,5.157,5.157,5.157,5.157,0
259969,LU1459802754,2022-03-11,16:52,15.905,15.905,15.905,15.905,64


### Get opening price per ISIN and day

In [9]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-03-10,08:00,12.240,12.240,12.240,12.240,5316,12.2400
1,DE000A0DJ6J9,2022-03-10,08:00,39.000,40.180,38.580,39.880,6489,39.0000
2,DE000A0D6554,2022-03-10,08:00,16.800,16.990,16.800,16.990,20802,16.8000
3,DE000A0D9PT0,2022-03-10,08:00,200.900,202.000,200.800,201.500,2727,200.9000
4,DE000A0HN5C6,2022-03-10,08:00,35.090,35.100,35.090,35.100,4470,35.0900
...,...,...,...,...,...,...,...,...,...
259966,DE000HAG0005,2022-03-11,16:44,23.800,23.800,23.800,23.800,97,22.9000
259967,DE000TUAG000,2022-03-11,16:45,2.757,2.757,2.757,2.757,3500,2.7070
259968,IE000WF4FCJ3,2022-03-11,16:46,5.157,5.157,5.157,5.157,0,5.2780
259969,LU1459802754,2022-03-11,16:52,15.905,15.905,15.905,15.905,64,15.4465


In [10]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-03-10,08:00,12.24,12.24,12.24,12.24,5316,12.24
156,AT0000A0E9W5,2022-03-10,08:01,12.25,12.28,12.25,12.28,744,12.24
3001,AT0000A0E9W5,2022-03-10,08:05,12.32,12.32,12.10,12.11,7844,12.24
3377,AT0000A0E9W5,2022-03-10,08:06,12.05,12.08,12.01,12.01,2130,12.24
3708,AT0000A0E9W5,2022-03-10,08:07,12.01,12.10,12.00,12.05,3795,12.24
...,...,...,...,...,...,...,...,...,...
254900,AT0000A0E9W5,2022-03-11,16:25,12.03,12.03,12.03,12.03,7,12.12
255532,AT0000A0E9W5,2022-03-11,16:27,12.06,12.06,12.04,12.04,571,12.12
255851,AT0000A0E9W5,2022-03-11,16:28,12.04,12.04,12.04,12.04,200,12.12
256219,AT0000A0E9W5,2022-03-11,16:29,12.03,12.06,12.03,12.03,239,12.12


### Get closing price per ISIN and day

In [11]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [12]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2022-03-10,08:00,12.24,12.24,12.24,12.24,5316,12.24,11.87
156,AT0000A0E9W5,2022-03-10,08:01,12.25,12.28,12.25,12.28,744,12.24,11.87
3001,AT0000A0E9W5,2022-03-10,08:05,12.32,12.32,12.10,12.11,7844,12.24,11.87
3377,AT0000A0E9W5,2022-03-10,08:06,12.05,12.08,12.01,12.01,2130,12.24,11.87
3708,AT0000A0E9W5,2022-03-10,08:07,12.01,12.10,12.00,12.05,3795,12.24,11.87
...,...,...,...,...,...,...,...,...,...,...
254900,AT0000A0E9W5,2022-03-11,16:25,12.03,12.03,12.03,12.03,7,12.12,12.01
255532,AT0000A0E9W5,2022-03-11,16:27,12.06,12.06,12.04,12.04,571,12.12,12.01
255851,AT0000A0E9W5,2022-03-11,16:28,12.04,12.04,12.04,12.04,200,12.12,12.01
256219,AT0000A0E9W5,2022-03-11,16:29,12.03,12.06,12.03,12.03,239,12.12,12.01


### Aggregations

In [13]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-03-10,35.8500,35.8500,35.8500,35.8500,0
1,AT000000STR1,2022-03-11,35.8000,35.4500,35.2500,35.8000,1265
2,AT00000FACC2,2022-03-10,7.4100,7.3500,7.3500,7.4100,200
3,AT00000FACC2,2022-03-11,7.4900,7.8400,7.4900,7.8400,716
4,AT0000606306,2022-03-10,14.1300,13.1100,12.8000,14.1300,64103
...,...,...,...,...,...,...,...
6424,XS2427474023,2022-03-11,23.1460,23.4500,23.1460,23.4500,0
6425,XS2434891219,2022-03-10,3.5456,3.5526,3.5388,3.5900,101700
6426,XS2434891219,2022-03-11,3.5644,3.5292,3.5292,3.6268,104385
6427,XS2437455608,2022-03-10,28.9200,28.9200,28.9200,28.9200,0


### Percent Change Prev Closing

In [14]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-03-10,35.8500,35.8500,35.8500,35.8500,0,NaN
1,AT000000STR1,2022-03-11,35.8000,35.4500,35.2500,35.8000,1265,35.8500
2,AT00000FACC2,2022-03-10,7.4100,7.3500,7.3500,7.4100,200,NaN
3,AT00000FACC2,2022-03-11,7.4900,7.8400,7.4900,7.8400,716,7.3500
4,AT0000606306,2022-03-10,14.1300,13.1100,12.8000,14.1300,64103,NaN
...,...,...,...,...,...,...,...,...
6424,XS2427474023,2022-03-11,23.1460,23.4500,23.1460,23.4500,0,23.2360
6425,XS2434891219,2022-03-10,3.5456,3.5526,3.5388,3.5900,101700,NaN
6426,XS2434891219,2022-03-11,3.5644,3.5292,3.5292,3.6268,104385,3.5526
6427,XS2437455608,2022-03-10,28.9200,28.9200,28.9200,28.9200,0,NaN


In [15]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [16]:
df_all.drop(columns=['prev_closing_price'], inplace=True)
df_all = df_all.round(decimals=2)

df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-03-10,35.85,35.85,35.85,35.85,0,NaN
1,AT000000STR1,2022-03-11,35.80,35.45,35.25,35.80,1265,-1.12
2,AT00000FACC2,2022-03-10,7.41,7.35,7.35,7.41,200,NaN
3,AT00000FACC2,2022-03-11,7.49,7.84,7.49,7.84,716,6.67
4,AT0000606306,2022-03-10,14.13,13.11,12.80,14.13,64103,NaN
...,...,...,...,...,...,...,...,...
6424,XS2427474023,2022-03-11,23.15,23.45,23.15,23.45,0,0.92
6425,XS2434891219,2022-03-10,3.55,3.55,3.54,3.59,101700,NaN
6426,XS2434891219,2022-03-11,3.56,3.53,3.53,3.63,104385,-0.66
6427,XS2437455608,2022-03-10,28.92,28.92,28.92,28.92,0,NaN


## Write to S3

In [17]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [18]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('python-etl-pipelines')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='python-etl-pipelines', key='xetra_daily_report_20220325_212722.parquet')

## Reading the above report from S3

In [19]:
bucket_target = s3.Bucket('python-etl-pipelines')

for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20220325_212722.parquet


In [20]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20220325_212722.parquet').get().get('Body').read()

data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-03-10,35.85,35.85,35.85,35.85,0,NaN
1,AT000000STR1,2022-03-11,35.80,35.45,35.25,35.80,1265,-1.12
2,AT00000FACC2,2022-03-10,7.41,7.35,7.35,7.41,200,NaN
3,AT00000FACC2,2022-03-11,7.49,7.84,7.49,7.84,716,6.67
4,AT0000606306,2022-03-10,14.13,13.11,12.80,14.13,64103,NaN
...,...,...,...,...,...,...,...,...
6424,XS2427474023,2022-03-11,23.15,23.45,23.15,23.45,0,0.92
6425,XS2434891219,2022-03-10,3.55,3.55,3.54,3.59,101700,NaN
6426,XS2434891219,2022-03-11,3.56,3.53,3.53,3.63,104385,-0.66
6427,XS2437455608,2022-03-10,28.92,28.92,28.92,28.92,0,NaN


## Done